In [ ]:
# 02_cleaning_merging.ipynb
#Project: Pedestrian Corridor Analysis — Cleaning & Merging  
#Author: Shohruzbek Abdumuminov and Ragib Asif
#Purpose: Parse WKT geometries, perform conservative cleaning of key columns, compute recent per-site summary metrics, spatially join count points to nearest corridor segments to inherit corridor `Category`, and save cleaned outputs into `data_clean/`.

In [2]:
# Imports
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (8,5)

# Create output folder
os.makedirs('data_clean', exist_ok=True)


In [12]:
# File paths
counts_path = "../data-raw/pedestrian_counts.csv"
demand_path = "../data-raw/pedestrian_demand.csv"

# Read CSVs (read as strings initially to avoid dtype inference issues)
counts = pd.read_csv(counts_path, dtype=str)
demand = pd.read_csv(demand_path, dtype=str)

print("Counts shape:", counts.shape)
print("Demand shape:", demand.shape)
# show columns
print("\nCounts columns:\n", counts.columns.tolist())
print("\nDemand columns:\n", demand.columns.tolist())


Counts shape: (114, 113)
Demand shape: (127277, 20)

Counts columns:
 ['the_geom', 'OBJECTID', 'Loc', 'Borough', 'Street_Nam', 'From_Stree', 'To_Street', 'Iex', 'May07_AM', 'May07_PM', 'May07_MD', 'Sept07_AM', 'Sept07_PM', 'Sept07_MD', 'May08_AM', 'May08_PM', 'May08_MD', 'Sept08_AM', 'Sept08_PM', 'Sept08_MD', 'May09_AM', 'May09_PM', 'May09_MD', 'Sept09_AM', 'Sept09_PM', 'Sept09_MD', 'May10_AM', 'May10_PM', 'May10_MD', 'Sept10_AM', 'Sept10_PM', 'Sept10_MD', 'May11_AM', 'May11_PM', 'May11_MD', 'Sept11_AM', 'Sept11_PM', 'Sept11_MD', 'May12_AM', 'May12_PM', 'May12_MD', 'Sept12_AM', 'Sept12_PM', 'Sept12_MD', 'May13_AM', 'May13_PM', 'May13_MD', 'Sept13_AM', 'Sept13_PM', 'Sept13_MD', 'May14_AM', 'May14_PM', 'May14_MD', 'Sept14_AM', 'Sept14_PM', 'Sept14_MD', 'May15_AM', 'May15_PM', 'May15_MD', 'Sept15_AM', 'Sept15_PM', 'Sept15_MD', 'May16_AM', 'May16_PM', 'May16_MD', 'Sept16_AM', 'Sept16_PM', 'Sept16_MD', 'May17_AM', 'May17_PM', 'May17_MD', 'Sept17_AM', 'Sept17_PM', 'Sept17_MD', 'May18_AM', 'M

In [13]:
# Parse WKT (shapely.wkt.loads) if 'the_geom' exists
if 'the_geom' in counts.columns:
    counts['geometry'] = counts['the_geom'].apply(lambda x: wkt.loads(x) if pd.notnull(x) else None)
else:
    raise ValueError("counts: 'the_geom' not found. Edit cell to use lat/lon if present.")

if 'the_geom' in demand.columns:
    demand['geometry'] = demand['the_geom'].apply(lambda x: wkt.loads(x) if pd.notnull(x) else None)
else:
    raise ValueError("demand: 'the_geom' not found. Ensure demand geometry exists.")

# Convert to GeoDataFrames with WGS84
gdf_counts = gpd.GeoDataFrame(counts.copy(), geometry='geometry', crs='EPSG:4326')
gdf_demand = gpd.GeoDataFrame(demand.copy(), geometry='geometry', crs='EPSG:4326')

print("gdf_counts:", gdf_counts.shape)
print("gdf_demand:", gdf_demand.shape)


gdf_counts: (114, 114)
gdf_demand: (127277, 21)


In [ ]:
#  helper
def clean_string_series(s):
    return s.astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

# Standardize borough-like columns
# counts sample uses 'Borough', demand sample uses 'BoroName' and 'Boro'
# Standardize to: Brooklyn, Queens, Manhattan, Staten Island, The Bronx
if 'Borough' in gdf_counts.columns:
    gdf_counts['Borough'] = clean_string_series(gdf_counts['Borough']).str.title()
    # Fix Staten Island (truncated as "Staten Isla")
    gdf_counts['Borough'] = gdf_counts['Borough'].replace('Staten Isla', 'Staten Island')
    # Fix Bronx to "The Bronx"
    gdf_counts['Borough'] = gdf_counts['Borough'].replace('Bronx', 'The Bronx')
    
    # Assign boroughs to bridge locations based on coordinates
    bridge_mask = gdf_counts['Borough'].isin(['East River Bridges', 'Harlem River Bridges'])
    if bridge_mask.sum() > 0:
        print(f"Found {bridge_mask.sum()} bridge locations. Assigning boroughs based on coordinates...")
        for idx in gdf_counts[bridge_mask].index:
            if gdf_counts.loc[idx, 'geometry'] is not None:
                lon = gdf_counts.loc[idx, 'geometry'].centroid.x
                lat = gdf_counts.loc[idx, 'geometry'].centroid.y
                street_name = gdf_counts.loc[idx, 'Street_Nam'] if 'Street_Nam' in gdf_counts.columns else ''
                assigned_borough = get_borough_from_coords(lon, lat, street_name)
                gdf_counts.loc[idx, 'Borough'] = assigned_borough
                print(f"  OBJECTID {gdf_counts.loc[idx, 'OBJECTID']}: {gdf_counts.loc[idx, 'Street_Nam']} -> {assigned_borough}")
    
    # Verify we have all 5 boroughs (no bridges should remain)
    valid_boroughs = ['Brooklyn', 'Queens', 'Manhattan', 'Staten Island', 'The Bronx']
    gdf_counts = gdf_counts[gdf_counts['Borough'].isin(valid_boroughs)].copy()
    print(f"\nAfter assigning bridge locations and filtering: {len(gdf_counts)} locations")
    print(f"Borough distribution: {gdf_counts['Borough'].value_counts().to_dict()}")
elif 'Boro' in gdf_counts.columns:
    gdf_counts['Borough'] = clean_string_series(gdf_counts['Boro']).str.title()
    gdf_counts['Borough'] = gdf_counts['Borough'].replace('Staten Isla', 'Staten Island')
    gdf_counts['Borough'] = gdf_counts['Borough'].replace('Bronx', 'The Bronx')
    
    # Assign boroughs to bridge locations based on coordinates
    bridge_mask = gdf_counts['Borough'].isin(['East River Bridges', 'Harlem River Bridges'])
    if bridge_mask.sum() > 0:
        print(f"Found {bridge_mask.sum()} bridge locations. Assigning boroughs based on coordinates...")
        for idx in gdf_counts[bridge_mask].index:
            if gdf_counts.loc[idx, 'geometry'] is not None:
                lon = gdf_counts.loc[idx, 'geometry'].centroid.x
                lat = gdf_counts.loc[idx, 'geometry'].centroid.y
                street_name = gdf_counts.loc[idx, 'Street_Nam'] if 'Street_Nam' in gdf_counts.columns else ''
                #assigned_borough = get_borough_from_coords(lon, lat, street_name)
                #gdf_counts.loc[idx, 'Borough'] = assigned_borough
                #print(f"  OBJECTID {gdf_counts.loc[idx, 'OBJECTID']}: {gdf_counts.loc[idx, 'Street_Nam']} -> {assigned_borough}")
    
    # Verify we have all 5 boroughs (no bridges should remain)
    valid_boroughs = ['Brooklyn', 'Queens', 'Manhattan', 'Staten Island', 'The Bronx']
    gdf_counts = gdf_counts[gdf_counts['Borough'].isin(valid_boroughs)].copy()
    print(f"\nAfter assigning bridge locations and filtering: {len(gdf_counts)} locations")
    print(f"Borough distribution: {gdf_counts['Borough'].value_counts().to_dict()}")

if 'BoroName' in gdf_demand.columns:
    gdf_demand['BoroName'] = clean_string_series(gdf_demand['BoroName']).str.title()
    # Handle multi-borough entries (e.g., "Manhattan,Brooklyn") - use first borough
    gdf_demand['BoroName'] = gdf_demand['BoroName'].str.split(',').str[0].str.strip()
    # Standardize to match counts
    gdf_demand['BoroName'] = gdf_demand['BoroName'].replace('Bronx', 'The Bronx')
    # Filter to only 5 main boroughs
    valid_boroughs = ['Brooklyn', 'Queens', 'Manhattan', 'Staten Island', 'The Bronx']
    gdf_demand = gdf_demand[gdf_demand['BoroName'].isin(valid_boroughs)].copy()
    print(f"After filtering demand to 5 boroughs: {len(gdf_demand)} segments")
elif 'Boro' in gdf_demand.columns:
    gdf_demand['BoroName'] = clean_string_series(gdf_demand['Boro']).str.title()
    gdf_demand['BoroName'] = gdf_demand['BoroName'].str.split(',').str[0].str.strip()
    gdf_demand['BoroName'] = gdf_demand['BoroName'].replace('Bronx', 'The Bronx')
    valid_boroughs = ['Brooklyn', 'Queens', 'Manhattan', 'Staten Island', 'The Bronx']
    gdf_demand = gdf_demand[gdf_demand['BoroName'].isin(valid_boroughs)].copy()

# Street name cleaning
# counts uses 'Street_Nam', demand uses 'street'
if 'Street_Nam' in gdf_counts.columns:
    gdf_counts['Street_Nam_clean'] = clean_string_series(gdf_counts['Street_Nam']).str.upper()
else:
    possible = [c for c in gdf_counts.columns if 'street' in c.lower()]
    if possible:
        gdf_counts['Street_Nam_clean'] = clean_string_series(gdf_counts[possible[0]]).str.upper()
    else:
        gdf_counts['Street_Nam_clean'] = ''

if 'street' in gdf_demand.columns:
    gdf_demand['street_clean'] = clean_string_series(gdf_demand['street']).str.upper()
else:
    possible = [c for c in gdf_demand.columns if 'street' in c.lower()]
    if possible:
        gdf_demand['street_clean'] = clean_string_series(gdf_demand[possible[0]]).str.upper()
    else:
        gdf_demand['street_clean'] = ''

# Category standardization (sample shows 'Category')
if 'Category' in gdf_demand.columns:
    gdf_demand['Category'] = clean_string_series(gdf_demand['Category']).str.title()

# Show samples
display(gdf_counts[['OBJECTID','Loc','Borough','Street_Nam','Street_Nam_clean']].head())
display(gdf_demand[['segmentid','BoroName','street','street_clean','Category']].head())


,OBJECTID,Loc,Borough,Street_Nam,Street_Nam_clean
0,1,1,Bronx,Broadway,BROADWAY
1,2,2,Bronx,East 161st Street,EAST 161ST STREET
2,3,3,Bronx,East Fordham Road,EAST FORDHAM ROAD
3,4,4,Bronx,East Gun Hill Road,EAST GUN HILL ROAD
4,5,5,Bronx,East Tremont Avenue,EAST TREMONT AVENUE


,segmentid,BoroName,street,street_clean,Category
0,0055131,Queens,122 STREET,122 STREET,Community
1,0000016,Staten Island,WARDS POINT AVENUE,WARDS POINT AVENUE,Baseline
2,0056031,Queens,120 STREET,120 STREET,Baseline
3,0180320,Queens,CROYDON ROAD,CROYDON ROAD,Baseline
4,0008394,Staten Island,MAPLE PARKWAY,MAPLE PARKWAY,Community


In [15]:
# Identify columns that look like count columns (heuristic: contain month tokens and am/pm/md)
tokens = ['may','oct','sept','jun','june','apr','mar','feb','jan','nov','dec']
count_cols = [c for c in gdf_counts.columns if any(t in c.lower() for t in tokens) and any(x in c.lower() for x in ['am','pm','md'])]
# fallback: detect columns with many numeric-looking entries
if not count_cols:
    numeric_candidates = []
    for c in gdf_counts.columns:
        # try to coerce some values
        sample = gdf_counts[c].dropna().astype(str).str.replace(',','').head(50)
        coerced = pd.to_numeric(sample, errors='coerce')
        if coerced.notna().sum() > 5:
            numeric_candidates.append(c)
    count_cols = numeric_candidates

print("Identified count columns (sample):", count_cols[:30])

# Convert to numeric (remove commas if present)
for c in count_cols:
    gdf_counts[c+'_num'] = pd.to_numeric(gdf_counts[c].str.replace(',',''), errors='coerce')

# List numeric columns created
num_cols = [c for c in gdf_counts.columns if c.endswith('_num')]
print("Numeric count columns:", num_cols[:30])


Identified count columns (sample): ['May07_AM', 'May07_PM', 'May07_MD', 'Sept07_AM', 'Sept07_PM', 'Sept07_MD', 'May08_AM', 'May08_PM', 'May08_MD', 'Sept08_AM', 'Sept08_PM', 'Sept08_MD', 'May09_AM', 'May09_PM', 'May09_MD', 'Sept09_AM', 'Sept09_PM', 'Sept09_MD', 'May10_AM', 'May10_PM', 'May10_MD', 'Sept10_AM', 'Sept10_PM', 'Sept10_MD', 'May11_AM', 'May11_PM', 'May11_MD', 'Sept11_AM', 'Sept11_PM', 'Sept11_MD']
Numeric count columns: ['May07_AM_num', 'May07_PM_num', 'May07_MD_num', 'Sept07_AM_num', 'Sept07_PM_num', 'Sept07_MD_num', 'May08_AM_num', 'May08_PM_num', 'May08_MD_num', 'Sept08_AM_num', 'Sept08_PM_num', 'Sept08_MD_num', 'May09_AM_num', 'May09_PM_num', 'May09_MD_num', 'Sept09_AM_num', 'Sept09_PM_num', 'Sept09_MD_num', 'May10_AM_num', 'May10_PM_num', 'May10_MD_num', 'Sept10_AM_num', 'Sept10_PM_num', 'Sept10_MD_num', 'May11_AM_num', 'May11_PM_num', 'May11_MD_num', 'Sept11_AM_num', 'Sept11_PM_num', 'Sept11_MD_num']


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: PerformanceWarning: DataFrame is highly fr

In [16]:
# Heuristic: prefer columns containing '24' or '25' as most recent
recent_cols = [c for c in num_cols if ('24' in c or '25' in c)]
if recent_cols:
    print("Using recent columns:", recent_cols)
    gdf_counts['avg_recent_count'] = gdf_counts[recent_cols].astype(float).mean(axis=1, skipna=True)
else:
    print("No explicit 24/25 columns found; using all numeric count cols for mean.")
    gdf_counts['avg_recent_count'] = gdf_counts[num_cols].astype(float).mean(axis=1, skipna=True)

# Examine results
display(gdf_counts[['OBJECTID','Loc','Street_Nam_clean','avg_recent_count']].head())
gdf_counts['avg_recent_count'].describe()


Using recent columns: ['June24_AM_num', 'June24_PM_num', 'June24_MD_num', 'Oct24_AM_num', 'Oct24_PM_num', 'Oct24_MD_num', 'May25_AM_num', 'May25_PM_num', 'May25_MD_num']


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


,OBJECTID,Loc,Street_Nam_clean,avg_recent_count
0,1,1,BROADWAY,2138.555556
1,2,2,EAST 161ST STREET,2964.555556
2,3,3,EAST FORDHAM ROAD,4018.111111
3,4,4,EAST GUN HILL ROAD,1535.777778
4,5,5,EAST TREMONT AVENUE,1370.111111


count      114.000000
mean      2934.093567
std       2659.981127
min          0.000000
25%       1136.305556
50%       2230.055556
75%       3908.750000
max      13352.333333
Name: avg_recent_count, dtype: float64

In [ ]:
# Project to metric CRS for distance calc (Web Mercator 3857)
gdf_counts = gdf_counts.set_geometry('geometry').to_crs(epsg=3857)
gdf_demand = gdf_demand.set_geometry('geometry').to_crs(epsg=3857)

# Keep minimal demand cols
demand_small = gdf_demand[['segmentid','street_clean','Category','geometry']].copy()

# sjoin_nearest to attach nearest corridor segment info
# geopandas.sjoin_nearest requires geopandas >= 0.10
joined = gpd.sjoin_nearest(gdf_counts, demand_small, how='left', distance_col='dist_m')

# Deduplicate: keep only the nearest match for each count location
# Some locations may match multiple segments at equal distance
joined = joined.sort_values('dist_m').drop_duplicates(subset=['OBJECTID'], keep='first')

print("Joined shape:", joined.shape)
print(f"Unique OBJECTID count: {joined['OBJECTID'].nunique()}")
display(joined[['OBJECTID','Loc','Street_Nam_clean','segmentid','street_clean','Category','dist_m']].head(10))


Joined shape: (155, 226)


,OBJECTID,Loc,Street_Nam_clean,segmentid,street_clean,Category,dist_m
0,1,1,BROADWAY,0079707,BROADWAY,Regional,0.125303
1,2,2,EAST 161ST STREET,0313939,EAST 161 STREET,Regional,1.417309
2,3,3,EAST FORDHAM ROAD,0080043,EAST FORDHAM ROAD,Regional,1.110133
3,4,4,EAST GUN HILL ROAD,0081299,EAST GUN HILL ROAD,Neighborhood,0.835461
4,5,5,EAST TREMONT AVENUE,0078822,EAST TREMONT AVENUE,Regional,0.329976
5,6,6,GRA CONCOURSE,0314755,GRAND CONCOURSE,Regional,0.330675
6,7,7,THIRD AVENUE,0070202,3 AVENUE,Neighborhood,1.364945
7,8,8,WHITE PLAINS ROAD,0087453,WHITE PLAINS ROAD,Neighborhood,1.818604
8,9,9,5TH AVENUE,0020712,5 AVENUE,Neighborhood,0.001692
9,10,10,5TH AVENUE,0022891,5 AVENUE,Regional,0.068242


In [ ]:
# How many matched to a Category?
joined['has_category'] = joined['Category'].notna()
pct_joined = joined['has_category'].mean() * 100
print(f"Percent of count sites matched to a corridor category: {pct_joined:.1f}%")

# Verify we have all 5 categories
all_categories = ['Global', 'Regional', 'Neighborhood', 'Community', 'Baseline']
categories_found = joined['Category'].unique()
print(f"\nCategories found: {sorted(categories_found)}")
print(f"Expected 5 categories: {all_categories}")
for cat in all_categories:
    count = (joined['Category'] == cat).sum()
    print(f"  {cat}: {count} locations")

# Verify we have all 5 boroughs
all_boroughs = ['Brooklyn', 'Queens', 'Manhattan', 'Staten Island', 'The Bronx']
boroughs_found = joined['Borough'].unique()
print(f"\nBoroughs found: {sorted(boroughs_found)}")
print(f"Expected 5 boroughs: {all_boroughs}")
for boro in all_boroughs:
    count = (joined['Borough'] == boro).sum()
    print(f"  {boro}: {count} locations")

# Distance distribution
display(joined['dist_m'].describe())

# Flag distant matches for manual review (e.g., > 100 m)
threshold_m = 100
joined['near_match_flag'] = joined['dist_m'] > threshold_m
display(joined[['OBJECTID','Loc','Borough','Category','dist_m','near_match_flag']].sort_values('dist_m').head(15))


Percent of count sites matched to a corridor category: 100.0%


count    155.000000
mean      42.384192
std      100.824071
min        0.000122
25%        0.031999
50%        0.383165
75%        4.215477
max      453.254574
Name: dist_m, dtype: float64

,OBJECTID,Loc,Category,dist_m,near_match_flag
10,11,11,Neighborhood,0.000122,False
63,64,64,Global,0.000454,False
63,64,64,Global,0.000454,False
33,34,34,Neighborhood,0.000563,False
16,17,17,Neighborhood,0.000683,False
68,69,69,Global,0.000700,False
64,65,65,Global,0.000835,False
67,68,68,Global,0.000945,False
67,68,68,Global,0.000945,False
93,94,94,Neighborhood,0.001109,False


In [ ]:
# Convert back to WGS84 for saving
out_gdf = joined.to_crs(epsg=4326)

# CSV of key columns
out_csv = 'data_clean/pedestrian_combined.csv'
cols_to_save = ['OBJECTID','Loc','Borough','Street_Nam_clean','avg_recent_count','segmentid','street_clean','Category','dist_m']
# Some columns might be missing if names differ; keep what exists
cols_to_save = [c for c in cols_to_save if c in out_gdf.columns]
out_gdf[cols_to_save].to_csv(out_csv, index=False)
print("Saved CSV:", out_csv)

# GeoJSON for mapping
out_geo = 'data_clean/pedestrian_combined.geojson'
out_gdf.to_file(out_geo, driver='GeoJSON')
print("Saved GeoJSON:", out_geo)
# End of notebook

## Next steps / Notes

- Inspect rows flagged with `near_match_flag == True`. If there are many, we can consider these:
  - Increasing threshold 
  - Using fuzzy name matching (thefuzz) comparing `Street_Nam_clean` with `street_clean`.
- If `avg_recent_count` is NaN for many sites, revisit `count_cols` detection and ensure numeric conversion succeeded.
- Commit `data_clean/pedestrian_combined.csv` to your `analysis` branch when validated.
- Later: load this CSV into SQL Server or push via SQLAlchemy to make it available to the dashboard backend.